<a href="https://colab.research.google.com/github/yonseimath/datascience-biginner-2022-kaggle-competitions/blob/feature%2Fyenakim/yenakim/AI4_Code_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
siz = 0.1  # size of validation set

splitter = GroupShuffleSplit(n_splits=1, test_size=siz, random_state=0)

# Split, keeping notebooks with a common origin (ancestor_id) together
ids = df.index.unique('id')
ancestors = df_ancestors.loc[ids, 'ancestor_id']
ids_train, ids_valid = next(splitter.split(ids, groups=ancestors))
ids_train, ids_valid = ids[ids_train], ids[ids_valid]

df_train = df.loc[ids_train, :]
df_valid = df.loc[ids_valid, :]

In [ ]:
df_train

In [ ]:
# Training set
tfidf = TfidfVectorizer(min_df=0.01)
X_train = tfidf.fit_transform(df_train['source'].astype(str))
# Rank of each cell within the notebook
y_train = df_ranks.loc[ids_train].to_numpy()
# Number of cells in each notebook
groups = df_ranks.loc[ids_train].groupby('id').size().to_numpy()

In [ ]:
print(X_train)

In [ ]:
# Add code cell ordering
X_train = sparse.hstack(( # 희소 행렬을 수평으로 쌓기
    X_train,
    np.where(
        df_train['cell_type'] == 'code',
        df_train.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1,
        0,
    ).reshape(-1, 1) # Code 셀이면 노트북 안 셀의 개수를, Markdown 셀이면 0을 넣는다
))
print(X_train.shape)

In [ ]:
model = XGBRanker(
    min_child_weight=10,
    subsample=0.5,
    tree_method='hist',
)
model.fit(X_train, y_train, group=groups)

In [ ]:
# Validation set
X_valid = tfidf.transform(df_valid['source'].astype(str))
# The metric uses cell ids
y_valid = df_orders.loc[ids_valid]

X_valid = sparse.hstack((
    X_valid,
    np.where(
        df_valid['cell_type'] == 'code',
        df_valid.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1,
        0,
    ).reshape(-1, 1)
))

In [ ]:
df_valid.index

In [ ]:
y_pred = pd.DataFrame({'rank': model.predict(X_valid)}, index=df_valid.index)
y_pred = (
    y_pred
    .sort_values(['id', 'rank'])  # Sort the cells in each notebook by their rank.
                                  # The cell_ids are now in the order the model predicted.
    .reset_index('cell_id')  # Convert the cell_id index into a column.
    .groupby('id')['cell_id'].apply(list)  # Group the cell_ids for each notebook into a list.
)
y_pred.head(10)

In [ ]:
df_valid.index.get_level_values('id')

In [ ]:
df_valid.index.get_level_values('id').unique()[8]

In [ ]:
nb_id = df_valid.index.get_level_values('id').unique()[8] # 잘 배열됐는지 확인

display(df.loc[nb_id])
display(df.loc[nb_id].loc[y_pred.loc[nb_id]])

In [ ]:
def count_inversions_slowly(ranks):
    inversions = 0
    size = len(ranks)
    for i in range(size):
        for j in range(i+1, size):
            if ranks[i] > ranks[j]:
                total += 1
    return total

def count_inversions(a):
    inversions = 0
    sorted_so_far = []
    for i, u in enumerate(a):  
        j = bisect(sorted_so_far, u)  
        inversions += i - j
        sorted_so_far.insert(j, u)  
    return inversions

# 켄달 상관계수
def kendall_tau(ground_truth, predictions):
    total_inversions = 0  
    total_2max = 0  
    for gt, pred in zip(ground_truth, predictions):
        ranks = [gt.index(x) for x in pred]  # rank predicted order in terms of ground truth
        total_inversions += count_inversions(ranks)
        n = len(gt)
        total_2max += n * (n - 1)
    return 1 - 4 * total_inversions / total_2max

In [ ]:
y_valid

In [ ]:
y_dummy

In [ ]:
y_dummy = df_valid.reset_index('cell_id').groupby('id')['cell_id'].apply(list)
kendall_tau(y_valid, y_dummy) # dummy : 정렬 X, valid : 정답

In [ ]:
kendall_tau(y_valid, y_pred) # pred : 예측

In [ ]:
paths = []
directory = '../input/AI4Code/test'
for file in os.scandir(directory):
    if file.is_file():
        paths.append(file.path)
    if len(paths) == num_train:
        break
        
id_names = []
for name in paths:
    name = name.split('/')
    id_n = name[-1].split('.')
    id_names.append(id_n[0])
    
# print(id_names)
# print(paths)

test_notebooks = []
for i in range(len(paths)):
    test_notebooks.append(read_notebook(paths[i],id_names[i]))
    
print(test_notebooks[0])

In [ ]:
# print(type(train_notebooks[0]))
df_test = (
    pd.concat(test_notebooks)
    .set_index('id', append=True)
    .swaplevel()
    .sort_index(level='id', sort_remaining=False)
)
df_test.head()

In [ ]:
X_test = tfidf.transform(df_test['source'].astype(str))
X_test = sparse.hstack((
    X_test,
    np.where(
        df_test['cell_type'] == 'code',
        df_test.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1,
        0,
    ).reshape(-1, 1)
))

In [ ]:
y_infer = pd.DataFrame({'rank': model.predict(X_test)}, index=df_test.index)
y_infer = y_infer.sort_values(['id', 'rank']).reset_index('cell_id').groupby('id')['cell_id'].apply(list)
y_infer

In [ ]:
y_submit = (
    y_infer
    .apply(' '.join)  # list of ids -> string of ids
    .rename_axis('id')
    .rename('cell_order')
)
y_submit

In [ ]:
y_submit.to_csv('submission.csv')